## Struktury danych

In [1]:
class Point:
    def __init__(self,x,y):
        self.x=x
        self.y=y
    def __str__(self):
        return "Point(%s,%s)"%(self.x,self.y)

In [2]:
class Segment:
    def __init__(self,p,q):
        
        if p.x < q.x:
            left, right=p, q
        else:
            left, right=q, p
                    
        self.left=left
        self.right=right
        self.a=(right.y-left.y)/(right.x-left.x) # zakladam ze nie ma pionowych odcinkow
        self.b=left.y-left.x*self.a
        
    def get_y(self,x):
        return self.a*x+self.b


In [3]:
class Trapezoid:
    def __init__(self, top, bottom, leftp, rightp):

        self.top = top
        self.bottom = bottom
        self.leftp = leftp
        self.rightp = rightp
        
        self.dnode = None # wskaznik do węzła w grafie wyszukiwań

        # wskazniki do sasiadow
        self.upper_left = None
        self.lower_left = None
        self.upper_right = None
        self.lower_right = None


In [28]:
class DNode:
    def __init__(self, node_type, label, left=None, right=None, above = None, below = None):
        """
        node_type: Typ węzła ('x', 'y' lub 'leaf')
        label: Punkt, odcinek lub trapez, zależnie od typu węzła
        left: Lewy potomek
        right: Prawy potomek
        """
        self.node_type = node_type  # Typ węzła: 'x', 'y', 'leaf'
        self.label = label          # Punkt, odcinek lub trapez
        self.left = left            # Lewy potomek
        self.right = right          # Prawy potomek
        self.above = above
        self.below = below
        self.parents=[]  # lista rodziców potrzebna do zastępowania trapezów w grafie wyszukiwań na inne węzły
        # potrzebne tylko w przypadku trapezów, ale nie chce mi się tego wydzielać do osobnej klasy
        
       
    def replace_node(self, new_node):
            # for parent in self.parents:
            #     if parent.left == self:
            #         parent.left = new_node
            #     elif parent.above == self:
            #         parent.above = new_node
            #     elif parent.below == self:
            #         parent.below = new_node
            #     else:
            #         parent.right = new_node
            #     new_node.parents.append(parent)
            self.node_type=new_node.node_type
            self.label=new_node.label
            self.left=new_node.left
            self.right=new_node.right
            self.above=new_node.above
            self.below=new_node.below

    
class DTree:
    def __init__(self):
        self.root = None

    def query(self, point,a=None):
        """
        Znajduje trapez, który zawiera dany punkt.
        point: (x, y) - współrzędne punktu
        Zwraca: Trapez (etykieta liścia), który zawiera punkt
        """
        current = self.root
        while current:
            if current.node_type == 'x':
                # Testujemy po współrzędnej x
                if point.x < current.label.x:
                    current = current.left                    
                else:
                    current = current.right

            elif current.node_type == 'y':
                # Testujemy położenie względem odcinka
                segment = current.label
                y_on_line = segment.get_y(point.x)
                if point.y > y_on_line:
                    current = current.above
                    
                elif point.y==y_on_line and a is not None:   # potrzebne do wyznaczania strefy dla s_i, jeżeli początek dodawanego odcinka leży na sprawdzanej linii
                    # nie wiem czy to rozwiazuje problem  
                    if segment.a>a:
                        current = current.below
                    else:
                        current = current.above
                else:
                    current = current.below

            elif current.node_type == 'leaf':
                # Znaleziony trapez
                return current.label

        return None

# Na wykładzie są podane dwa rodzaje tej struktury - musimy wytłumaczyć dlaczego akurat tą wybierami?

In [ ]:
# Testy struktury DTree
if __name__ == "__main__":
    # Tworzenie przykładowych danych
    p1 = Point(1, 1)
    p2 = Point(5, 5)
    p3 = Point(3, 2)
    segment = Segment(p1, p2)

    trapez1 = Trapezoid(None, None, p1, p2)
    trapez2 = Trapezoid(None, None, p2, p3)

    leaf1 = DNode('leaf', trapez1)
    leaf2 = DNode('leaf', trapez2)
    x_node = DNode('x', p2, leaf1, leaf2)

    # Tworzenie drzewa
    dtree = DTree()
    dtree.root = x_node

    # Test punktu
    test_point = Point(4, 4)
    result = dtree.query(test_point)
    print("Trapez zawierający punkt:", result.leftp.x, result.rightp.x)

## Generator losowych odcinków
Wylosowane odcinki są w położeniu ogólnym i spełniają założenia algorytmu

In [ ]:
import random

def segments_intersect(s1, s2):

    def ccw(a, b, c):
        """Sprawdza orientację (a, b, c): czy jest zgodna z ruchem wskazówek zegara."""
        return (c[1] - a[1]) * (b[0] - a[0]) - (b[1] - a[1]) * (c[0] - a[0])>1e-10
    
    (a1, a2), (b1, b2) = s1, s2
    return (ccw(a1, b1, b2) != ccw(a2, b1, b2)) and (ccw(a1, a2, b1) != ccw(a1, a2, b2))

def segment_contains_point(s, p):
    """
    Sprawdza, czy odcinek s zawiera punkt p (punkt p może leżeć na końcu odcinka, dlatego wtedy funkcja zwraca false).
    """
    (x1, y1), (x2, y2) = s
    if p>(min(x1,x2),min(y1,y2)) and p<(max(x1,x2),max(y1,y2)) and abs((x1-x2)*(p[1]-y1)-(y1-y2)*(p[0]-x1))<1e-10: # sprawdzam czy punkt leży na prostej zawierającej odcinek
        return True
    return False

def generate_segments(n, x_range, y_range):

    segments = []

    for _ in range(n):
        while True:
            x1 = random.uniform(x_range[0], x_range[1])
            y1 = random.uniform(y_range[0], y_range[1])
            x2 = random.uniform(x_range[0], x_range[1])
            y2 = random.uniform(y_range[0], y_range[1])

            if x1== x2:
                continue
            
            new_segment = ((x1, y1), (x2, y2))
            
            if all(not segments_intersect(new_segment, segment) for segment in segments) and all(not segment_contains_point(segment, (x1, y1)) and not segment_contains_point(segment, (x2, y2)) for segment in segments):
                segments.append(new_segment)
                break

    return segments

# Przykład użycia
n = 20
x_range = (0, 10)
y_range = (0, 10)
lines = generate_segments(n, x_range, y_range)
for segment in segments:
    print(segment)
    
import matplotlib.pyplot as plt
plt.figure()
for segment in lines:
    (x1, y1), (x2, y2) = segment
    plt.plot([x1, x2], [y1, y2], 'o-')
plt.show()


## Aplikacja do zadawania odcinków

Aby wprowadzić odcinek należy nacisnąć lewym przyciskiem myszy w punkcie początkowym, przeciągnąć kursor i zwolnić przycisk w punkcie końcowym.
Po wprowadzeniu wszystkich odcinków wystarczy zamknąć okno.

In [72]:
%matplotlib tk

import matplotlib.pyplot as plt

class LineDrawer:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.lines = []
        self.current_line = None
        self.start_point = None

        # stałe wymiary okna (można zmienić)
        self.ax.set_xlim(0, 10)
        self.ax.set_ylim(0, 10)

        self.cid_click = self.fig.canvas.mpl_connect('button_press_event', self.on_click)
        self.cid_motion = self.fig.canvas.mpl_connect('motion_notify_event', self.on_motion)
        self.cid_release = self.fig.canvas.mpl_connect('button_release_event', self.on_release)

    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        self.start_point = (event.xdata, event.ydata)
        self.ax.plot(event.xdata, event.ydata, 'ko') 
        self.current_line = self.ax.plot([event.xdata, event.xdata], [event.ydata, event.ydata], 'k-')[0]

    def on_motion(self, event):
        if self.current_line is None or event.inaxes != self.ax:
            return
        xdata, ydata = self.current_line.get_data()
        xdata[1], ydata[1] = event.xdata, event.ydata
        self.current_line.set_data(xdata, ydata)
        self.fig.canvas.draw()

    def on_release(self, event):
        if self.current_line is None or event.inaxes != self.ax:
            return
        self.lines.append((self.start_point, (event.xdata, event.ydata)))
        self.ax.plot(event.xdata, event.ydata, 'ko') 
        self.current_line = None
        self.start_point = None
        self.fig.canvas.draw()

    def show(self):
        plt.show()

    def get_lines(self):
        return self.lines

drawer = LineDrawer()
drawer.show()
lines=drawer.get_lines()


In [ ]:
print(lines)

In [32]:
# zamiana reprezentacji odcinków
def change_segment_representation(lines):
    s=[0 for _ in range(len(lines))]
    for i in range(len(lines)):
        p=Point(lines[i][0][0],lines[i][0][1])
        q=Point(lines[i][1][0],lines[i][1][1])
        segment=Segment(p,q)
        s[i]=segment
    return s

In [80]:

segments = change_segment_representation(lines)

## Konstrukcja mapy trapezowej T(S)

wyznaczanie strefy dla $s_i$

In [34]:
def find_area(D,segment):
    area=[]
    p=segment.left
    q=segment.right
    delta=D.query(p,segment.a)
    area.append(delta)
    while q.x>delta.rightp.x:
        if delta.rightp.y>segment.get_y(delta.rightp.x):
            area.append(delta.lower_right)
            delta=delta.lower_right
        else:
            area.append(delta.upper_right)
            delta=delta.upper_right
    return area
        



In [35]:

def update_left(old_trap, left, top, bottom):
    if left:
        if old_trap.upper_left:
            if old_trap.upper_left.upper_right == old_trap:
                old_trap.upper_left.upper_right = left
            if old_trap.upper_left.lower_right == old_trap:
                old_trap.upper_left.lower_right = left
        if old_trap.lower_left:
            if old_trap.lower_left.upper_right == old_trap:
                old_trap.lower_left.upper_right = left
            if old_trap.lower_left.lower_right == old_trap:
                old_trap.lower_left.lower_right = left
    else:
        if old_trap.upper_left:
            old_trap.upper_left.upper_right = top
        if old_trap.lower_left:
            old_trap.lower_left.lower_right = bottom


In [36]:
def update_right(old_trap, right, top, bottom):
    if right:
        if old_trap.upper_right:
            if old_trap.upper_right.upper_left == old_trap:
                old_trap.upper_right.upper_left = right
            if old_trap.upper_right.lower_left == old_trap:
                old_trap.upper_right.lower_left = right
        if old_trap.lower_right:
            if old_trap.lower_right.upper_left == old_trap:
                old_trap.lower_right.upper_left = right
            if old_trap.lower_right.lower_left == old_trap:
                old_trap.lower_right.lower_left = right
    else:
        if old_trap.upper_right:
            old_trap.upper_right.upper_left = top
        if old_trap.lower_right:
            old_trap.lower_right.lower_left = bottom

In [37]:
def insert_into_one_trapezoid(dtree, segment, trapezoid):
    # Funkcja która wstawia odcinek do jednego trapezu
    added_traps=set()

    if segment.left.x > trapezoid.leftp.x:
        left_trapezoid = Trapezoid(trapezoid.top, trapezoid.bottom, trapezoid.leftp, segment.left)
        added_traps.add(left_trapezoid)
    else:
        left_trapezoid = None
    
    if segment.right.x < trapezoid.rightp.x:
        right_trapezoid = Trapezoid(trapezoid.top, trapezoid.bottom, segment.right, trapezoid.rightp)
        added_traps.add(right_trapezoid)
    else:
        right_trapezoid = None
        
    top_trapezoid = Trapezoid(trapezoid.top, segment, segment.left, segment.right)
    bottom_trapezoid = Trapezoid(segment, trapezoid.bottom, segment.left, segment.right)
    added_traps.add(top_trapezoid)
    added_traps.add(bottom_trapezoid)

    
    if left_trapezoid:
        left_trapezoid.upper_right = top_trapezoid
        left_trapezoid.lower_right = bottom_trapezoid
        left_trapezoid.upper_left=trapezoid.upper_left
        left_trapezoid.lower_left=trapezoid.lower_left
        top_trapezoid.upper_left = left_trapezoid
        bottom_trapezoid.lower_left = left_trapezoid
        
    else:
        top_trapezoid.upper_left = trapezoid.upper_left
        bottom_trapezoid.lower_left = trapezoid.lower_left
        
    update_left(trapezoid, left_trapezoid, top_trapezoid, bottom_trapezoid)

    
    if right_trapezoid:
        right_trapezoid.upper_left = top_trapezoid
        right_trapezoid.lower_left = bottom_trapezoid
        right_trapezoid.upper_right=trapezoid.upper_right
        right_trapezoid.lower_right=trapezoid.lower_right
        top_trapezoid.upper_right = right_trapezoid
        bottom_trapezoid.lower_right = right_trapezoid

    else:
        top_trapezoid.upper_right = trapezoid.upper_right
        bottom_trapezoid.lower_right = trapezoid.lower_right

    update_right(trapezoid, right_trapezoid, top_trapezoid, bottom_trapezoid)
            

    # zmiany w dtree
    leftpoint=DNode('x',segment.left)
    if left_trapezoid:
        left=left_trapezoid
    else:
        if trapezoid.upper_left:
            left=trapezoid.upper_left
        else:
            left=trapezoid.lower_left
    leftpoint.left=DNode('leaf',left)
    if left_trapezoid:
        left.dnode=leftpoint.left
    if left:
        left.dnode.parents.append(leftpoint)
    rightpoint=DNode('x',segment.right)
    leftpoint.right=rightpoint
    if right_trapezoid:
        right=right_trapezoid
    else:
        if trapezoid.upper_right:
            right=trapezoid.upper_right
        else:
            right=trapezoid.lower_right
    rightpoint.right=DNode('leaf',right)
    if right_trapezoid:
        right.dnode=rightpoint.right
    if right:
        right.dnode.parents.append(rightpoint)
    segment_node=DNode('y',segment)
    rightpoint.left=segment_node
    
    segment_node.above=DNode('leaf',top_trapezoid)
    top_trapezoid.dnode=segment_node.above
    top_trapezoid.dnode.parents.append(segment_node)
    
    segment_node.below=DNode('leaf',bottom_trapezoid)
    bottom_trapezoid.dnode=segment_node.below
    bottom_trapezoid.dnode.parents.append(segment_node)
    if dtree.root==trapezoid.dnode:
        dtree.root=leftpoint
    else:
        trapezoid.dnode.replace_node(leftpoint)
    return added_traps
    
    


In [38]:
def main(lines):
    segments = shuffle_segments(change_segment_representation(lines))
    
    outer_trapezoid = outer_trapezoid(lines)
    
    # Tworzenie drzewa
    dtree = DTree()
    # Tworzenie korzenia
    x_node = DNode('leaf', outer_trapezoid)
    outer_trapezoid.dnode = x_node
    dtree.root = x_node
    # zbiór trapezów należących do mapy
    T=set([outer_trapezoid])
    
    # segments = change_segment_representation(segments)
    
    #lecimy po kolei po odcinkach, po kolei je analizujemy
    for segment in lines:
        
        area=find_area(dtree,segment)
        #patrzymy ile trapezow zajmuje
        n = len( area )
        if n ==1 :
            #tutaj funkcja
            insert_into_one_trapezoid(dtree,segment,area[0])
            pass
        elif n > 1 :
            #tutaj funkcja
            pass
    return dtree
        


In [39]:
def outer_trapezoid(segments):
    # Inicjalizujemy zmienne, które będą przechowywać minimalne i maksymalne wartości
    min_x = float('inf')
    max_x = float('-inf')
    min_y = float('inf')
    max_y = float('-inf')

    # Przechodzimy przez wszystkie odcinki, aby znaleźć ekstremalne punkty
    for (x1, y1), (x2, y2) in segments:
        min_x = min(min_x, x1, x2)
        max_x = max(max_x, x1, x2)
        min_y = min(min_y, y1, y2)
        max_y = max(max_y, y1, y2)

    # Trapez będzie miał wierzchołki w skrajnych punktach
    # trapezoid = [(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)]
    upper_segment=Segment(Point(min_x-1, max_y+1),Point(max_x+1, max_y+1))
    lower_segment=Segment(Point(min_x-1, min_y-1),Point(max_x+1, min_y-1))
    trapezoid=Trapezoid(upper_segment,lower_segment,Point(min_x-1, min_y-1),Point(max_x+1, max_y+1))
    
    return trapezoid


In [40]:
# korzysta z innej reprezentacji trapezow i odcinkow
import matplotlib.pyplot as plt
def visualize_segments_and_trapezoid(segments):
    # Tworzymy trapez obejmujący wszystkie odcinki
    trapezoid = outer_trapezoid(segments)
    
    # Rysujemy odcinki
    plt.figure(figsize=(8, 6))
    for (x1, y1), (x2, y2) in segments:
        plt.plot([x1, x2], [y1, y2], marker='o', color='b', label='Odcinek' if (x1, y1) == segments[0][0] else "")

    trapezoid_x = [trapezoid.leftp.x, trapezoid.rightp.x, trapezoid.rightp.x, trapezoid.leftp.x, trapezoid.leftp.x]
    trapezoid_y = [trapezoid.top.get_y(trapezoid.leftp.x), trapezoid.top.get_y(trapezoid.rightp.x), trapezoid.bottom.get_y(trapezoid.rightp.x), trapezoid.bottom.get_y(trapezoid.leftp.x), trapezoid.top.get_y(trapezoid.leftp.x)]
    plt.plot(trapezoid_x, trapezoid_y, color='r', label='Trapez', linestyle='--')
    
    # Dodatkowe ustawienia wykresu
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Wizualizacja odcinków i trapezu')
    plt.grid(True)
    plt.legend()
    plt.show()
visualize_segments_and_trapezoid(lines)


        

In [41]:
import random

def shuffle_segments(segments):
    random.shuffle(segments)
    return segments

In [ ]:
# test czy funkcje sie uruchamiaja
T = DTree()
outer_trap = outer_trapezoid(lines)
node = DNode('leaf', outer_trap)
outer_trap.dnode = node  # Ensure the dnode attribute is set
T.root = node


# Test insert_into_one_trapezoid

insert_into_one_trapezoid(T, segments[0], outer_trap)
print(T.root.node_type)
print(T.root.left.label.leftp.x)
print(T.root.right.node_type)
print(T.root.right.left.node_type)
print(T.root.right.right.label.leftp.x)
print(T.root.right.left.above.label.leftp.x)
print(T.root.right.left.below.label.leftp.x)


In [43]:
def insertSegmentInManyTrapezoids(searchGraph, intersectingTrapezoids, segment):

    upperMidTrapezoid = None
    lowerMidTrapezoid = None
    mergeUpper = False

    addedTrapezoids = []
    
    for trapezoid in intersectingTrapezoids:

        if trapezoid == intersectingTrapezoids[0]:

            leftTrapezoid = Trapezoid(trapezoid.top, trapezoid.bottom, trapezoid.leftp, segment.left)
            if trapezoid.rightp.y > segment.get_y(trapezoid.rightp.x):
                upperMidTrapezoid = Trapezoid(trapezoid.top, segment, segment.left, trapezoid.rightp)
                lowerMidTrapezoid = Trapezoid(segment, trapezoid.bottom, segment.left, trapezoid.rightp) 
                mergeUpper = False
            else:
                upperMidTrapezoid = Trapezoid(trapezoid.top, segment, segment.left, trapezoid.rightp) 
                lowerMidTrapezoid = Trapezoid(segment, trapezoid.bottom, segment.left, trapezoid.rightp)
                mergeUpper = True
                
            leftTrapezoid.upper_right = upperMidTrapezoid
            leftTrapezoid.upper_left = trapezoid.upper_left
            leftTrapezoid.lower_right = lowerMidTrapezoid
            leftTrapezoid.lower_left = trapezoid.lower_left
            
            
            if mergeUpper:
                upperMidTrapezoid.upper_right = trapezoid.upper_right
                upperMidTrapezoid.lower_right = trapezoid.upper_right
                lowerMidTrapezoid.upper_right = trapezoid.upper_right 
                lowerMidTrapezoid.lower_right = trapezoid.lower_right
            else:
                upperMidTrapezoid.upper_right = trapezoid.upper_right
                upperMidTrapezoid.lower_right = trapezoid.lower_right
                lowerMidTrapezoid.upper_right = trapezoid.lower_right
                lowerMidTrapezoid.lower_right = trapezoid.lower_right
            
            upperMidTrapezoid.upper_left = leftTrapezoid
            upperMidTrapezoid.lower_left = leftTrapezoid
            
            lowerMidTrapezoid.upper_left= leftTrapezoid
            lowerMidTrapezoid.lower_left = leftTrapezoid
            
            #updateLeftNeighbors(trapezoid, leftTrapezoid)

            update_left(trapezoid, leftTrapezoid, upperMidTrapezoid, lowerMidTrapezoid)
            if mergeUpper:
                #updateRightNeighbors(trapezoid, lowerMidTrapezoid)
                update_right(trapezoid, None, None, lowerMidTrapezoid) #zmienione
            else:
                #updateRightNeighbors(trapezoid, upperMidTrapezoid)
                update_right(trapezoid,None,upperMidTrapezoid, None) #zmienione

            addedTrapezoids.append(leftTrapezoid)
            addedTrapezoids.append(upperMidTrapezoid)
            addedTrapezoids.append(lowerMidTrapezoid)

            leftTrapezoidNode = DNode('leaf', leftTrapezoid)
            leftTrapezoid.dnode = leftTrapezoidNode
            upperMidTrapezoidNode = DNode('leaf', upperMidTrapezoid)
            upperMidTrapezoid.dnode = upperMidTrapezoidNode
            lowerMidTrapezoidNode = DNode('leaf', lowerMidTrapezoid)
            lowerMidTrapezoid.dnode = lowerMidTrapezoidNode
                
            sNode = DNode('y', segment, above = upperMidTrapezoidNode, below =  lowerMidTrapezoidNode)
            
            pNode = DNode('x', segment.left, left = leftTrapezoidNode, right = sNode)
            
            leftTrapezoidNode.parents.append(pNode)
            upperMidTrapezoidNode.parents.append(sNode)
            lowerMidTrapezoidNode.parents.append(sNode)
            
            #trapezoid.node.replaceNode(searchGraph, pNode)
            trapezoid.dnode.replace_node(pNode)

        elif trapezoid == intersectingTrapezoids[-1]:

            rightTrapezoid = Trapezoid(trapezoid.top, trapezoid.bottom, segment.right, trapezoid.rightp)
            if mergeUpper:
                upperMidTrapezoid.rightp = segment.right
                lowerMidTrapezoid = Trapezoid(segment, trapezoid.bottom, trapezoid.leftp, segment.right)
                upperMidTrapezoidNode = upperMidTrapezoid.dnode
                upperMidTrapezoid.dnode=upperMidTrapezoidNode
                lowerMidTrapezoidNode = DNode('leaf', lowerMidTrapezoid)
                lowerMidTrapezoid.dnode=lowerMidTrapezoidNode
            else:
                upperMidTrapezoid = Trapezoid(trapezoid.top, segment, trapezoid.leftp, segment.right)
                lowerMidTrapezoid.rightp = segment.right
                upperMidTrapezoidNode = DNode('leaf', upperMidTrapezoid)
                upperMidTrapezoid.dnode=upperMidTrapezoidNode
                lowerMidTrapezoidNode = lowerMidTrapezoid.dnode
                lowerMidTrapezoid.dnode=lowerMidTrapezoidNode
                
            rightTrapezoid.upper_right = trapezoid.upper_right
            rightTrapezoid.upper_left = upperMidTrapezoid
            rightTrapezoid.lower_right = trapezoid.lower_right
            rightTrapezoid.lower_left = lowerMidTrapezoid

            upperMidTrapezoid.upper_right = rightTrapezoid
            upperMidTrapezoid.lower_right = rightTrapezoid
            lowerMidTrapezoid.upper_right = rightTrapezoid
            lowerMidTrapezoid.lower_right = rightTrapezoid
            
            #sprawdz czy tego nie można usunąć!
            # if mergeUpper:
            #     lowerMidTrapezoid.upper_left = trapezoid.upper_left
            #     lowerMidTrapezoid.lower_left = trapezoid.lower_left
            # else:
            #     upperMidTrapezoid.upper_left = trapezoid.upper_left
            #     upperMidTrapezoid.lower_left = trapezoid.lower_left
            
            #updateRightNeighbors(trapezoid, rightTrapezoid)
            update_right(trapezoid, rightTrapezoid, upperMidTrapezoid, lowerMidTrapezoid)
            if mergeUpper:
                #updateLeftNeighbors(trapezoid, lowerMidTrapezoid)
                update_left(trapezoid, None,None, lowerMidTrapezoid)
            else:
                #updateLeftNeighbors(trapezoid, upperMidTrapezoid) 
                update_left(trapezoid, None, upperMidTrapezoid,None)
            
            addedTrapezoids.append(upperMidTrapezoid)
            addedTrapezoids.append(lowerMidTrapezoid)
            addedTrapezoids.append(rightTrapezoid)
                
            rightTrapezoidNode = DNode('leaf', rightTrapezoid)
            rightTrapezoid.dnode = rightTrapezoidNode
            
            sNode = DNode('y', segment, above = upperMidTrapezoidNode, below =  lowerMidTrapezoidNode)
            upperMidTrapezoidNode.parents.append(sNode)
            lowerMidTrapezoidNode.parents.append(sNode)
            
            qNode = DNode('x', segment.right, left = sNode, right = rightTrapezoidNode)
            rightTrapezoidNode.parents.append(qNode)
            
            trapezoid.dnode.replace_node(qNode)
            #trapezoid.node.replaceNode(searchGraph, qNode)

        else:
            
            change = False

            if mergeUpper:
                oldLowerMidTrapezoid = lowerMidTrapezoid
                lowerMidTrapezoid = Trapezoid(segment, trapezoid.bottom, trapezoid.leftp, trapezoid.rightp) 
                upperMidTrapezoidNode = upperMidTrapezoid.dnode
                upperMidTrapezoid.dnode=upperMidTrapezoidNode
                lowerMidTrapezoidNode = DNode('leaf', lowerMidTrapezoid)
                lowerMidTrapezoid.dnode=lowerMidTrapezoidNode
            else:
                oldUpperMidTrapezoid = upperMidTrapezoid
                upperMidTrapezoid = Trapezoid(trapezoid.top, segment, trapezoid.leftp, trapezoid.rightp) 
                upperMidTrapezoidNode = DNode('leaf', upperMidTrapezoid)
                upperMidTrapezoid.dnode=upperMidTrapezoidNode
                lowerMidTrapezoidNode = lowerMidTrapezoid.dnode
                lowerMidTrapezoid.dnode=lowerMidTrapezoidNode

            if trapezoid.rightp.y > segment.get_y(trapezoid.rightp.x):
                upperMidTrapezoid.rightp = trapezoid.rightp
                if mergeUpper == True:
                    change = True
                mergeUpper = False
                addedTrapezoids.append(upperMidTrapezoid)
            else:
                lowerMidTrapezoid.rightp = trapezoid.rightp
                if mergeUpper == False:
                    change = True
                mergeUpper = True
                addedTrapezoids.append(lowerMidTrapezoid)
                
            if mergeUpper:
                if change:
                    upperMidTrapezoid.upper_right = trapezoid.upper_right
                    upperMidTrapezoid.lower_right = trapezoid.upper_right
                    upperMidTrapezoid.upper_left = trapezoid.upper_left
                    upperMidTrapezoid.lower_left = oldUpperMidTrapezoid

                    lowerMidTrapezoid.upper_right = trapezoid.upper_right
                    lowerMidTrapezoid.lower_right = trapezoid.lower_right
                    
                    #updateLeftNeighbors(trapezoid, upperMidTrapezoid)
                    update_left(trapezoid, None, upperMidTrapezoid,None)
                    #updateRightNeighbors(trapezoid, lowerMidTrapezoid)
                    update_right(trapezoid,None,None, lowerMidTrapezoid)
                else:
                    upperMidTrapezoid.upper_right = trapezoid.upper_right
                    upperMidTrapezoid.lower_right = trapezoid.upper_right

                    lowerMidTrapezoid.upper_right = trapezoid.upper_right
                    lowerMidTrapezoid.lower_right = trapezoid.lower_right
                    lowerMidTrapezoid.upper_left = oldLowerMidTrapezoid
                    lowerMidTrapezoid.lower_left = trapezoid.lower_left
                    
                    #updateLeftNeighbors(trapezoid, lowerMidTrapezoid)
                    update_left(trapezoid, None,None, lowerMidTrapezoid)
                    #updateRightNeighbors(trapezoid, lowerMidTrapezoid)  
                    update_right(trapezoid, None,None, lowerMidTrapezoid)
            else:
                if change:
                    upperMidTrapezoid.upper_right = trapezoid.upper_right
                    upperMidTrapezoid.lower_right = trapezoid.lower_right

                    lowerMidTrapezoid.upper_right = trapezoid.lower_right
                    lowerMidTrapezoid.lower_right = trapezoid.lower_right
                    lowerMidTrapezoid.upper_left = oldLowerMidTrapezoid
                    lowerMidTrapezoid.lower_left = trapezoid.lower_left
                    
                    #updateLeftNeighbors(trapezoid, lowerMidTrapezoid)
                    update_left(trapezoid, None,None, lowerMidTrapezoid)
                    #updateRightNeighbors(trapezoid, upperMidTrapezoid)
                    update_right(trapezoid, None, upperMidTrapezoid, None)
                else:
                    upperMidTrapezoid.upper_right = trapezoid.upper_right
                    upperMidTrapezoid.upper_left = trapezoid.upper_left
                    upperMidTrapezoid.lower_right = trapezoid.lower_right
                    upperMidTrapezoid.lower_left = oldUpperMidTrapezoid

                    lowerMidTrapezoid.upper_right = trapezoid.lower_right
                    lowerMidTrapezoid.lower_right = trapezoid.lower_right
                    
                    #updateLeftNeighbors(trapezoid, upperMidTrapezoid)
                    update_left(trapezoid, None, upperMidTrapezoid, None)
                    #updateRightNeighbors(trapezoid, upperMidTrapezoid)
                    update_right(trapezoid, None, upperMidTrapezoid, None)


            sNode = DNode('y', segment, above = upperMidTrapezoidNode, below =  lowerMidTrapezoidNode)
            upperMidTrapezoidNode.parents.append(sNode)
            lowerMidTrapezoidNode.parents.append(sNode)
            trapezoid.dnode.replace_node(sNode)
            
    uniqueTrapezoids = set(addedTrapezoids)     
    return uniqueTrapezoids

In [ ]:
def updateTreeMany(trapezoids, segment, newTrapezoidsAbove, newTrapezoidsBelow, left, right):
    node = trapezoids[0].node
    if left:
        node.type = 'x'
        node.label = segment.left
        lnode = DNode('tnode', left)
        node.left = lnode
        left.node = lnode
        rnode = DNode('snode', segment)
        node.right = rnode
        rlnode = DNode('tnode', newTrapezoidsAbove[0])
        rnode.left = rlnode
        newTrapezoidsAbove[0].node = rlnode
        rrnode = DNode('tnode', newTrapezoidsBelow[0])
        rnode.right = rrnode
        newTrapezoidsBelow[0].node = rrnode
    else:
        node.type = 'snode'
        node.label = segment
        lnode = DNode('tnode', newTrapezoidsAbove[0])
        node.left = lnode
        newTrapezoidsAbove[0].node = lnode
        rnode = DNode('tnode', newTrapezoidsBelow[0])
        node.right = rnode
        newTrapezoidsBelow[0].node = rnode

    i = 0
    j = 0
    k = len(newTrapezoidsAbove)
    m = len(newTrapezoidsBelow)

    while i + j < len(trapezoids) - 2:
        node = trapezoids[i + j + 1].node
        node.type = 'snode'
        node.label = segment

        if newTrapezoidsAbove[i].rightPoint > newTrapezoidsBelow[j].rightPoint or i == k - 1:
            j += 1
            lnode = newTrapezoidsAbove[i].node
            node.left = lnode
            rnode = DNode('tnode', newTrapezoidsBelow[j])
            node.right = rnode
            newTrapezoidsBelow[j].node = rnode
        else:
            i += 1
            lnode = DNode('tnode', newTrapezoidsAbove[i])
            node.left = lnode
            newTrapezoidsAbove[i].node = lnode
            rnode = newTrapezoidsBelow[j].node
            node.right = rnode

    node = trapezoids[-1].node
    if right:
        node.type = 'x'
        node.label = segment.right
        rnode = DNode('tnode', right)
        node.right = rnode
        right.node = rnode
        lnode = DNode('snode', segment)
        node.left = lnode
        lastNode = lnode
    else:
        node.type = 'snode'
        node.label = segment
        lastNode = node

    if i == k - 1 and j == m - 1:
        lastNode.left = newTrapezoidsAbove[i].node
        lastNode.right = newTrapezoidsBelow[j].node
    elif j == m - 1:
        lnode = DNode('tnode', newTrapezoidsAbove[-1])
        lastNode.left = lnode
        newTrapezoidsAbove[-1].node = lnode
        lastNode.right = newTrapezoidsBelow[j].node
    else:
        rnode = DNode('tnode', newTrapezoidsBelow[-1])
        lastNode.right = rnode
        newTrapezoidsBelow[-1].node = rnode
        lastNode.left = newTrapezoidsAbove[i].node

In [ ]:
def insertIntoMany (T, trapezoids: list[Trapezoid], segment: Segment):
    p, q = segment.left, segment.right
    left, right = None, None
    newTrapezoidsAbove = []
    newTrapezoidsBelow = []
    n = len(trapezoids)
    first = trapezoids[0]
    lower_left = first.lower_left
    upper_left = first.upper_left
    upper_right = first.upper_right
    lower_right = first.lower_right
    x=first.rightp.x
    
    if first.rightp>segment.get_y(x):
        top = Trapezoid(first.top, segment, p, first.rightp)
        bottom = Trapezoid(segment, first.bottom, p, Point(x, segment.get_y(x)))
        merge = "lower"
    else:
        top = Trapezoid(first.top, segment, p, Point(x, segment.get_y(x)))
        bottom = Trapezoid(segment, first.bottom, p, first.rightp)
        merge = "upper"
        
    if first.leftp < p:
        left = Trapezoid(first.top, first.bottom, first.leftp, p)
        left.lower_left = lower_left
        left.upper_left = upper_left
        left.upper_right = top
        left.lower_right = bottom
        top.upper_left = left
        bottom.lower_left = left
        update_left(first, left)    
    else:           
        bottom.lower_left = lower_left
        top.upper_left = upper_left
        if lower_left:
            lower_left.lower_right = bottom
            if lower_left.top > segment:
                lower_left.upper_right = top
                top.lower_left = lower_left
        if upper_left:
            upper_left.upper_right = top
            if segment > upper_left.bottom:
                upper_left.lower_right = bottom
                bottom.upper_left = upper_left
                             
    if trapezoids[1] == first.lower_right:
        top.upper_right = upper_right
        if upper_right:
            upper_right.upper_left = top
    else:
        bottom.lower_right = lower_right
        if lower_right:
            lower_right.lower_left = bottom
    
    newTrapezoidsAbove.append(top)
    newTrapezoidsBelow.append(bottom)
    
    for i in range(1, n-1):
        nextPoint = trapezoids[i].rightp
        lower_left = trapezoids[i].lower_left
        upper_left = trapezoids[i].upper_left
        upper_right = trapezoids[i].upper_right
        lower_right = trapezoids[i].lower_right
        x=nextPoint.x
        
        if segment.isAbove(nextPoint):
            lowerrightp = Point(x, segment.getY(x))
            if merge == "upper":
                top.rightp = nextPoint
                t = Trapezoid(segment, trapezoids[i].bottom, bottom.rightp, lowerrightp)
                bottom.upper_right = t
                t.upper_left = bottom
                t.lower_left = lower_left
                if lower_left:
                    lower_left.lower_right = t
                top.upper_right = upper_right
                if upper_right:
                    upper_right.upper_left = top
                bottom = t
                newTrapezoidsBelow.append(bottom)
            else:
                bottom.rightp = lowerrightp
                t = Trapezoid(trapezoids[i].top, segment, top.rightp, nextPoint)
                top.lower_right = t
                t.lower_left = top
                t.upper_left = upper_left
                t.upper_right = upper_right
                if upper_left:
                    upper_left.upper_right = t
                if upper_right:
                    upper_right.upper_left = t
                top = t
                newTrapezoidsAbove.append(top)
            merge = "lower"   
        else:
            upperrightp = Point(x, segment.getY(x))
            if merge == "upper":
                top.rightp = upperrightp
                t = Trapezoid(segment, trapezoids[i].bottom, bottom.rightp, nextPoint)
                bottom.upper_right = t
                t.upper_left = bottom
                t.lower_left = lower_left
                t.lower_right = lower_right
                if lower_left:
                    lower_left.lower_right = t
                if lower_right:
                    lower_right.lower_left = t
                bottom = t
                newTrapezoidsBelow.append(bottom)
            else:
                bottom.rightp = nextPoint
                t = Trapezoid(trapezoids[i].top, segment, top.rightp, upperrightp)
                top.lower_right = t
                t.lower_left = top
                t.upper_left = upper_left
                if upper_left:
                    upper_left.upper_right = t
                bottom.lower_right = lower_right
                if lower_right:
                    lower_right.lower_left = bottom
                top = t
                newTrapezoidsAbove.append(top)
            merge="upper"
            
    last = trapezoids[n-1]
    lower_left = last.lower_left
    upper_left = last.upper_left
    upper_right = last.upper_right
    lower_right = last.lower_right
    x=last.rightp
        
    if merge == "upper":
        top.rightp = q
        t = Trapezoid(segment, last.bottom, bottom.rightp, q)
        bottom.upper_right = t
        t.upper_left = bottom
        bottom = t
        newTrapezoidsBelow.append(bottom)
    else:
        bottom.rightp = q
        t = Trapezoid(last.top, segment, top.rightp, q)
        top.lower_right = t
        t.lower_left = top
        top = t
        newTrapezoidsAbove.append(top)
        
    if last.rightp > q:
        right = Trapezoid(last.top, last.bottom, q, last.rightp)
        right.lower_left = bottom
        right.upper_left = top
        right.upper_right = upper_right
        right.lower_right = lower_right
        top.upper_right = right
        bottom.lower_right = right
        update_right(last, right)
    else:
        top.upper_right = upper_right
        bottom.lower_right = lower_right
        if lower_right:
            lower_right.lower_left = bottom
            if lower_right.top > segment:
                lower_right.upper_left = top
                top.lower_right = lower_right
        if upper_right:
            upper_right.upper_left = top
            if segment > upper_right.bottom:
                upper_right.lower_left = bottom
                bottom.upper_right = upper_right
                
    if last == trapezoids[n-2].upper_right:
        bottom.lower_left = lower_left
        if lower_left:
            lower_left.lower_right = bottom
    else:
        top.upper_left = upper_left
        if upper_left:
            upper_left.upper_right = top
            
    updateTreeMany(trapezoids, segment, newTrapezoidsAbove, newTrapezoidsBelow, left, right)

In [44]:
def visualise_map_construction():
    
    o_trapezoid = outer_trapezoid(lines)
    shuffle_segments(segments)

    # Funkcja pomocnicza do rysowania trapezoidów
    def plot_trapezoid(trapezoid, color='g', linestyle='-'):
        plt.plot([trapezoid.leftp.x, trapezoid.leftp.x], [trapezoid.bottom.get_y(trapezoid.leftp.x), trapezoid.top.get_y(trapezoid.leftp.x)], color=color,linestyle=linestyle)
        plt.plot([trapezoid.rightp.x, trapezoid.rightp.x], [trapezoid.bottom.get_y(trapezoid.rightp.x), trapezoid.top.get_y(trapezoid.rightp.x)], color=color,linestyle=linestyle)

    def draw_outer_trapezoid():
        o_trapezoid_x = [o_trapezoid.leftp.x, o_trapezoid.rightp.x, o_trapezoid.rightp.x, o_trapezoid.leftp.x, o_trapezoid.leftp.x]
        o_trapezoid_y = [o_trapezoid.top.get_y(o_trapezoid.leftp.x), o_trapezoid.top.get_y(o_trapezoid.rightp.x), o_trapezoid.bottom.get_y(o_trapezoid.rightp.x), o_trapezoid.bottom.get_y(o_trapezoid.leftp.x), o_trapezoid.top.get_y(o_trapezoid.leftp.x)]
        plt.plot(o_trapezoid_x, o_trapezoid_y, color='g', linestyle='-')
    
    # Inicjalizacja wykresu
    plt.figure() 
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Wizualizacja etapów konstrukcji mapy')
    draw_outer_trapezoid()

    # Tworzenie drzewa
    dtree = DTree()
    # Tworzenie korzenia
    x_node = DNode('leaf', o_trapezoid)
    o_trapezoid.dnode = x_node
    dtree.root = x_node
    # Zbiór trapezów należących do mapy
    T = set([o_trapezoid])
    S=set()
    
    for segment in segments:

        area = find_area(dtree, segment)
        
        # Aktualizacja wykresu po wstawieniu segmentu
        plt.draw()
        plt.pause(1)  # Pauza, aby zobaczyć zmiany

        # Patrzymy ile trapezow zajmuje
        n = len(area)
        if n == 1:
            new=insert_into_one_trapezoid(dtree, segment, area[0])
        elif n > 1:
            new=insertSegmentInManyTrapezoids(dtree, area, segment)
        
        # Rysowanie segmentu
        plt.plot([segment.left.x, segment.right.x], [segment.left.y, segment.right.y],marker='o', color='r')
        plt.pause(1)
        # Usunięcie poprzedniego wykresu i rysowanie nowych trapezoidów po wstawieniu segmentu
        plt.clf()
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('Wizualizacja etapów konstrukcji mapy')
        # Rysowanie segmentu
        plt.plot([segment.left.x, segment.right.x], [segment.left.y, segment.right.y],marker='o', color='r')
        draw_outer_trapezoid()

        T-=set(area)
        T|=new
        for trapezoid in T:
            plot_trapezoid(trapezoid, color='g')
        for s in S:
            plt.plot([s.left.x, s.right.x], [s.left.y, s.right.y],marker='o', color='b')
        plt.draw()
        plt.pause(1)  # Pauza, aby zobaczyć zmiany
        plt.plot([segment.left.x, segment.right.x], [segment.left.y, segment.right.y], marker='o',color='b')
        S.add(segment)

    # Końcowe wyświetlenie
    draw_outer_trapezoid()
    plt.show()




In [81]:

visualise_map_construction()